# 3. Spark

Spark Programming Guide: <https://spark.apache.org/docs/latest/> (use Python API recommended)
Spark API: <https://spark.apache.org/docs/latest/api/python/index.html>


# 3.1 Example Walkthrough
3.1 Follow the Spark Examples below! After completion see Exercise 3.2 and 3.3!


### Initialize PySpark

First, we use the findspark package to initialize PySpark.

In [1]:
# Initialize PySpark
#SPARK_MASTER="local[1]"
SPARK_MASTER="spark://mpp3r01c03s03.cos.lrz.de:7077"
APP_NAME = "PySpark Lecture Herget"

# If there is no SparkSession, create the environment
try:
    sc and spark
except NameError as e:
  #import findspark
  #findspark.init()
    import pyspark
    import pyspark.sql
    conf=pyspark.SparkConf().set("spark.cores.max", "4")
    sc = pyspark.SparkContext(master=SPARK_MASTER, conf=conf)
    spark = pyspark.sql.SparkSession(sc).builder.appName(APP_NAME).getOrCreate()

print("PySpark initiated...")

Error:  name 'sc' is not defined
PySpark initiated...


### Hello, World!

Loading data, mapping it and collecting the records into RAM...

In [11]:
# Load the text file using the SparkContext
csv_lines = sc.textFile("../data/example.csv")

# Map the data to split the lines into a list
data = csv_lines.map(lambda line: line.split(","))

# Collect the dataset into local RAM
data.collect()

[['Russell Jurney', 'Relato', 'CEO'],
 ['Florian Liebert', 'Mesosphere', 'CEO'],
 ['Don Brown', 'Rocana', 'CIO'],
 ['Steve Jobs', 'Apple', 'CEO'],
 ['Donald Trump', 'The Trump Organization', 'CEO'],
 ['Russell Jurney', 'Data Syndrome', 'Principal Consultant']]

### Creating Objects from CSV

Using a function with a map operation to create objects (dicts) as records...

In [3]:
# Turn the CSV lines into objects
def csv_to_record(line):
    parts = line.split(",")
    record = {
      "name": parts[0],
      "company": parts[1],
      "title": parts[2]
    }
    return record

# Apply the function to every record
records = csv_lines.map(csv_to_record)

# Inspect the first item in the dataset
records.first()

{'company': 'Relato', 'name': 'Russell Jurney', 'title': 'CEO'}

### GroupBy

Using the groupBy operator to count the number of jobs per person...

In [4]:
# Group the records by the name of the person
grouped_records = records.groupBy(lambda x: x["name"])

# Show the first group
grouped_records.first()

# Count the groups
job_counts = grouped_records.map(
  lambda x: {
    "name": x[0],
    "job_count": len(x[1])
  }
)

job_counts.first()

job_counts.collect()

[{'job_count': 1, 'name': 'Florian Liebert'},
 {'job_count': 1, 'name': 'Donald Trump'},
 {'job_count': 2, 'name': 'Russell Jurney'},
 {'job_count': 1, 'name': 'Don Brown'},
 {'job_count': 1, 'name': 'Steve Jobs'}]

### Map vs FlatMap

Understanding the difference between the map and flatmap operators...

In [5]:
# Compute a relation of words by line
words_by_line = csv_lines\
  .map(lambda line: line.split(","))

print(words_by_line.collect())

# Compute a relation of words
flattened_words = csv_lines\
  .map(lambda line: line.split(","))\
  .flatMap(lambda x: x)

flattened_words.collect()

[[u'Russell Jurney', u'Relato', u'CEO'], [u'Florian Liebert', u'Mesosphere', u'CEO'], [u'Don Brown', u'Rocana', u'CIO'], [u'Steve Jobs', u'Apple', u'CEO'], [u'Donald Trump', u'The Trump Organization', u'CEO'], [u'Russell Jurney', u'Data Syndrome', u'Principal Consultant']]


[u'Russell Jurney',
 u'Relato',
 u'CEO',
 u'Florian Liebert',
 u'Mesosphere',
 u'CEO',
 u'Don Brown',
 u'Rocana',
 u'CIO',
 u'Steve Jobs',
 u'Apple',
 u'CEO',
 u'Donald Trump',
 u'The Trump Organization',
 u'CEO',
 u'Russell Jurney',
 u'Data Syndrome',
 u'Principal Consultant']

---
## Further Exercises

3.2 Implement a wordcount using Spark. Who many words are in the file `example.csv`?


In [103]:
csv_lines = sc.textFile("../data/example.csv")
count = csv_lines.flatMap(lambda line: line.split(',')).flatMap(lambda line: line.split(' ')) 
count2 = csv_lines.flatMap(lambda line: line.split(',')).flatMap(lambda line: line.split(' '))\
                  .map(lambda word: (word, 1)).reduceByKey(lambda x,y: x+y)
count.count()

28

In [104]:
count2.take(5)

[('Trump', 2), ('The', 1), ('CEO', 4), ('Russell', 2), ('Syndrome', 1)]

3.3 How many log enteries per HTTP Response Code exist? 

In [115]:
def nasa_split_http(line):
    parts = line.split(' ')
    if (len(parts) > 2):
        return ''+parts[-2]
    else:
        return '0'

nasa = sc.textFile("../data/nasa/NASA_access_log_Jul95")
nasa_count = nasa.flatMap(lambda line: line.split('\n')).map(nasa_split_http).map(lambda word: (word, 1)).reduceByKey(lambda x,y: x+y)
nasa_count.collect()

[('304', 132627),
 ('404', 10845),
 ('200', 1701534),
 ('302', 46573),
 ('501', 14),
 ('400', 5),
 ('500', 62),
 ('403', 54),
 ('0', 1)]